# Imports

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session

# Connect to Snowflake

In [3]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [4]:
# Create some test data to work with
sentences_german = [
    "Künstliche Intelligenz revolutioniert zunehmend verschiedenste Branchen, von der Automobilindustrie bis zur Medizintechnik.",
    "Unternehmen investieren vermehrt in Cloud-Technologien, um ihre Daten effizienter zu verwalten und zu sichern.",
    "Die Entwicklung nachhaltiger Technologien wird als Schlüssel zur Bekämpfung des Klimawandels angesehen.",
    "Im Bereich der Robotik werden Fortschritte gemacht, die es Maschinen ermöglichen, komplexere Aufgaben zu übernehmen.",
    "Die neueste Generation von Smartphones bietet verbesserte Kamerafunktionen und längere Akkulaufzeiten.",
    "Datenschutz und Privatsphäre im Internet gewinnen durch neue gesetzliche Regelungen an Bedeutung.",
    "Virtual Reality erlebt einen Boom, mit Anwendungen, die von Bildung bis Unterhaltung reichen.",
    "Autonome Fahrzeuge werden intensiv getestet, und es wird erwartet, dass sie die Art und Weise, wie wir reisen, verändern werden.",
    "Wearables werden zunehmend smarter und finden Anwendung in Gesundheitsüberwachung und Fitness.",
    "3D-Drucktechnologie entwickelt sich weiter und wird bereits in der Produktion von Bauteilen bis hin zu ganzen Gebäuden eingesetzt."
]


df = session.create_dataframe(sentences_german, schema=['INPUTS'])
df.show(n=15, max_width=1000)

--------------------------------------------------------------------------------------------------------------------------------------
|"INPUTS"                                                                                                                            |
--------------------------------------------------------------------------------------------------------------------------------------
|Künstliche Intelligenz revolutioniert zunehmend verschiedenste Branchen, von der Automobilindustrie bis zur Medizintechnik.         |
|Unternehmen investieren vermehrt in Cloud-Technologien, um ihre Daten effizienter zu verwalten und zu sichern.                      |
|Die Entwicklung nachhaltiger Technologien wird als Schlüssel zur Bekämpfung des Klimawandels angesehen.                             |
|Im Bereich der Robotik werden Fortschritte gemacht, die es Maschinen ermöglichen, komplexere Aufgaben zu übernehmen.                |
|Die neueste Generation von Smartphones bietet verbesse

# Register & Run Translation Model

In [5]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [6]:
# Get the translation model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en")

# Register the model to Snowflake
snow_model_custom = reg.log_model(
    pipe, 
    model_name='opus_mt_de_en', 
    conda_dependencies=['tokenizers','transformers','sentencepiece']
    )

# Model signature
translations = snow_model_custom.run(df).cache_result()
translations.show(n=10, max_width=1000)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]], 'forced_eos_token_id': 0}


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"INPUTS"                                                                                                                            |"TRANSLATION_TEXT"                                                                                                                       |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Die neueste Generation von Smartphones bietet verbesserte Kamerafunktionen und längere Akkulaufzeiten.                              |The latest generation of smartphones offers imp